# Precondition SkinT_test

In [41]:
import pandas as pd

In [42]:
ID_arr = [1,2,3,4,5,6]
csv_file_arr = ["1","2","3","4"]
day_arr = [1,2]
set = [4,5,6]

date_dict = {
    4: {
        1:"19",
        2:"20"
    },
    5: {
        1:"24",
        2:"25"
    },
    6: {
        1:"26",
        2:"27"
    },
}

In [ ]:
for set_ in set:
    with pd.ExcelWriter(f'SkinT_set{set_}.xlsx') as writer:        
        for id_ in ID_arr:
            result = pd.DataFrame(columns=["Time","Unit","1","2","3","4","Tmsk"])
            for day_ in day_arr:
                if set_==4 and day_==2 and id_==1:
                    continue
                result_temp = pd.DataFrame(columns=["Time","Unit","1","2","3","4","Tmsk"])
                for csv_ in csv_file_arr:
                    file_name = f"database/Set {set_}/Day{day_}/ID{id_}-{csv_}_07{date_dict[set_][day_]}23.csv"
                    print("Process on:",file_name)
                    data_read = pd.read_csv(
                        file_name, skiprows=19
                    )
                    result_temp['Time'] = data_read['Date/Time']
                    result_temp['Unit'] = data_read['Unit']
                    result_temp[csv_] = data_read['Value']
                result_temp['Tmsk'] = 0.3*result_temp["1"]+0.3*result_temp["2"]+0.2*result_temp["3"]+0.2*result_temp["4"]
                frames = [result, result_temp]
                result = pd.concat(frames)
            result.to_excel(writer, sheet_name=f"id{id_}", index=False)

# Insert the mean data

In [44]:
master_data = pd.read_excel("Experiment_edited_V4.xlsx")
slice_data = master_data[['End Date',"Experiment set",'Experiment id', "Time_thermal"]]
slice_data.head()

,End Date,Experiment set,Experiment id,Time_thermal
0,2023-07-10 11:02:50,set1,id2,10:45-11:05
1,2023-07-10 11:03:41,set1,id6,10:45-11:05
2,2023-07-10 11:03:52,set1,id1,10:45-11:05
3,2023-07-10 11:04:08,set1,id5,10:45-11:05
4,2023-07-10 11:05:05,set1,id3,10:45-11:05


In [138]:
data_set = {}
new_set = [1,2,3,4,5,6]
for set_ in new_set:
    data_set["set"+str(set_)]={}
    for id_ in ID_arr:
        data_set["set"+str(set_)]["id"+str(id_)] = pd.read_excel(f"SkinT_set{set_}.xlsx", sheet_name=f"id{id_}", parse_dates=['Time'], date_format="mixed")

In [139]:
from datetime import datetime
refineset = ["set1","set2"]
for set_ in refineset:
    for id_ in ID_arr:
        date_result = []
        # print(f"process on {set_} id{id_}")
        for _, row in data_set[set_]["id"+str(id_)].iterrows():
            date_ = row['Time']
            if date_.day == 13:
                date_result.append(date_)
                continue    
            date_ = datetime.strptime(str(date_), "%Y-%d-%m %H:%M:%S")
            date_result.append(date_)
        data_set[set_]["id"+str(id_)].Time=date_result

process on set1 id1
process on set1 id2
process on set1 id3
process on set1 id4
process on set1 id5
process on set1 id6
process on set2 id1
process on set2 id2
process on set2 id3
process on set2 id4
process on set2 id5
process on set2 id6


In [140]:
from datetime import datetime
# end_time = datetime.strptime("2023-07-10 11:05", "%Y-%m-%d %H:%M")
start_time = datetime.strptime("2023-07-10 10:45", "%Y-%m-%d %H:%M")
data_set["set1"]["id1"][data_set["set1"]["id1"]['Time']>=start_time]

,Time,Unit,1,2,3,4,Tmsk
46,2023-07-10 10:45:01,C,35.095,34.098,34.595,33.140,34.3049
47,2023-07-10 10:46:01,C,35.095,33.599,34.595,33.140,34.1552
48,2023-07-10 10:47:01,C,35.095,33.100,34.595,33.140,34.0055
49,2023-07-10 10:48:01,C,35.095,33.100,35.095,33.140,34.1055
50,2023-07-10 10:49:01,C,34.596,33.599,35.095,33.140,34.1055
...,...,...,...,...,...,...,...
979,2023-07-11 18:10:01,C,25.105,25.105,25.104,25.148,25.1134
980,2023-07-11 18:11:01,C,25.105,25.105,25.104,25.148,25.1134
981,2023-07-11 18:12:01,C,25.105,25.105,25.104,25.148,25.1134
982,2023-07-11 18:13:01,C,25.105,25.105,25.104,25.148,25.1134


In [141]:
from datetime import datetime
result_tmsk_avg = []
try:
    for _, row in slice_data.iterrows():
        date = str(row["End Date"].date())
        start_clock, end_clock = row['Time_thermal'].split("-")
        start_time = date+" "+start_clock
        end_time = date+" "+end_clock

        end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")

        # print(row)
        # print(start_time)
        # print(end_time)

        dataFiltered = data_set[row['Experiment set']][row['Experiment id']][(end_time > data_set[row['Experiment set']][row['Experiment id']]['Time']) & (start_time <= data_set[row['Experiment set']][row['Experiment id']]['Time'])]
        # print(dataFiltered)
        # break
        result_tmsk_avg.append(dataFiltered.Tmsk.mean(axis=0))
except ValueError:
    print(row) 

master_data['Average_tmsk'] = result_tmsk_avg

In [142]:
master_data.to_excel("test.xlsx", index=False)